In [1]:
max_n_image = 192560
max_n_image_not_nan = 42560

In [2]:
from Model import SegNetModel, RetrainedClassificationModel
from HyperparameterOptimization import findOptimalBatchSize

Using TensorFlow backend.


In [3]:
from math import log, ceil
import numpy as np
import keras

In [ ]:
from keras.utils import plot_model
model = SegNetModel('noname')
plot_model(model.model, to_file='model.png')

In [ ]:
model = SegNetModel('noname')
bs = findOptimalBatchSize(model, 2, save_excel=False)
model.train(bs, n_epoch=100000)
model.model.save('maxsized.hd5')
acc = model.eval(bs, remove_nan=False)
print(reduce_size, acc)

In [ ]:
!pip install pydot

In [ ]:
model = SegNetModel('noname')
bs = findOptimalBatchSize(model, 2, save_excel=False, reduced_size=1024)
acc_arr = []
hst_arr = []
for reduce_size in np.array(2) ** range(11, int(ceil(log(max_n_image_not_nan, 2))) + 1):
    reduce_size = min(reduce_size, max_n_image_not_nan)
    hst = model.train(bs, n_epoch=100000, reduced_size=reduce_size)
    hst_arr.append(hst)
    model.model.save(str(reduce_size) + '.hd5')
    acc = model.eval(bs, remove_nan=False)
    acc_arr.append(acc)
    print(reduce_size, acc)

In [ ]:
hst = model.train(bs, n_epoch=1000, remove_nan=False)
model.model.save('alldata.hd5')
acc.append(model.eval(bs, remove_nan=False))